# Hardpoint Breakaway Tests - Recent History

This notebook evaluates the hardpoint breakaway tests performed on a given `day_obs`.  
The most important values to evaluate per hardpoint are:
  
  1. The measured forces when the breakaway happens in the positive direction (compression).
  2. The measured forces when the breakawat happens in the negative direction (tension).
  3. The stiffness of the breakaway mechanism.
     This is defined as the linear coefficient of the polynomial that fits the
     displacement versus the measured forces. 

In [ ]:
# Times Square Parameters
day_obs = 20250804  # YYYYMMDD

In [ ]:
import asyncio
import ipywidgets as W
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from astropy.time import Time
from typing import Optional
from io import BytesIO
from IPython.display import display, clear_output
from typing import List

from lsst_efd_client import EfdClient
from lsst.ts.xml.enums.MTM1M3 import HardpointTest
from lsst.summit.utils.efdUtils import (
    getEfdData,
    getDayObsEndTime,
    getDayObsStartTime,
    makeEfdClient,
)

In [ ]:
# --- Global Variables ---
# HP BreakAway Limits from https://sitcomtn-082.lsst.io/
COMPRESSION_LOWER_LIMIT = 2981  # N
COMPRESSION_UPPER_LIMIT = 3959  # N
TENSION_LOWER_LIMIT = -4420  # N
TENSION_UPPER_LIMIT = -3456  # N

# Used to select the window size near zero measured forces.
DISPLACEMENT_CROP_RANGE = 300  # um
DISPLACEMENT_CROP_RANGE_FOR_FIT = 100  # um

# Used to determine if a test was valid or not
MEASURED_FORCE_MAXIMUM_TOLERANCE = 1000  # N

# Meter to micrometer
METER_TO_MICROMETER = 1e6  # um/m

# Maximum spec stiffness
SPEC_STIFFNESS = 100  # N/um

# Status Plotting
STATUS_COLORS = {
    HardpointTest.NOTTESTED: "lightgrey",
    HardpointTest.MOVINGNEGATIVE: "lightsteelblue",
    HardpointTest.TESTINGPOSITIVE: "forestgreen",
    HardpointTest.TESTINGNEGATIVE: "royalblue",
    HardpointTest.MOVINGREFERENCE: "darkseagreen",
    HardpointTest.PASSED: "dimgrey",
    HardpointTest.FAILED: "red",
}

# We use this a lot, so let's make it shorter
TEST_POS_COLOR = STATUS_COLORS[HardpointTest.TESTINGPOSITIVE]
TEST_NEG_COLOR = STATUS_COLORS[HardpointTest.TESTINGNEGATIVE]

# Number of hardpoints
n_hardpoints = 6

# Gap size to split different test runs
gap_size = "3min"

# Create an EFD client instance
efd_client = makeEfdClient()

# Get the start and end times
start_time = getDayObsStartTime(day_obs)
end_time = getDayObsEndTime(day_obs)

In [ ]:
# Query the CSC level command that we can use to define the start of a
# breakaway test for a given strut.
test_command_df = getEfdData(
    client=efd_client,
    topic="lsst.sal.MTM1M3.command_testHardpoint",
    columns=["hardpointActuator"],
    begin=start_time,
    end=end_time,
)


# test_command_df = await efd_client.select_time_series(
#     topic_name="lsst.sal.MTM1M3.command_testHardpoint",
#     fields=["hardpointActuator"],
#     start=start_time,
#     end=end_time
# )

In [ ]:
# Query the status of the breakaway tests as they evolve.
test_status_df = getEfdData(
    client=efd_client,
    topic="lsst.sal.MTM1M3.logevent_hardpointTestStatus",
    columns=[f"testState{i}" for i in range(n_hardpoints)],
    begin=start_time,
    end=end_time,
)

# test_command_df = await efd_client.select_time_series(
#     topic_name="lsst.sal.MTM1M3.logevent_hardpointTestStatus",
#     fields=[f"testState{i}" for i in range(n_hardpoints)],
#     start=start_time,
#     end=end_time
# )